In [ ]:
%matplotlib qt
import lmfit
import numpy as np
from bluesky import RunEngine
from bluesky.plans import scan, rel_scan, list_scan, rel_list_scan, log_scan
from bluesky.plan_stubs import mv
from bluesky.callbacks import LiveFitPlot, LiveFit, LivePlot
from bluesky.callbacks.fitting import PeakStats
from bluesky.callbacks.mpl_plotting import plot_peak_stats
from bluesky.callbacks.best_effort import BestEffortCallback
from ophyd.sim import det, motor, noisy_det
from matplotlib.pyplot import ion, subplots
from bluesky.utils import install_nb_kicker
install_nb_kicker()

ion()

In [ ]:
RE = RunEngine()

bec = BestEffortCallback()
RE.subscribe(bec)

motor.delay = 0.2

## Scan

In [ ]:
help(scan)

In [ ]:
RE(
    scan([det], motor, -1, 1, num=20)
)

In [ ]:
RE(
    scan([det], motor, -5, 5, num=20)
)

In [ ]:
RE(
    scan([det], motor, -5, 5, num=100)
)

## Plot fitting

In [ ]:
fig, ax = subplots()
bec.disable_plots()
def gaussian(x, peak, sigma, x0):
    return peak*np.exp(-(x - x0)**2/(2 * sigma**2))

model = lmfit.Model(gaussian)
init_guess = { 'peak': 2, 'sigma': lmfit.Parameter('sigma', 3, min=0), 'x0': 0}
live_fit = LiveFit(model, 'noisy_det', {'x': 'motor'}, init_guess)
live_fit_plot = LiveFitPlot(live_fit, color='r', ax=ax)
live_plot = LivePlot('noisy_det', 'motor', marker='x', linestyle='none', ax=ax)
RE(
    scan([noisy_det], motor, -5, 5, num=40),
    [live_fit_plot, live_plot]
)

### Peak Stats

* cen: center of mass
* com: mid-point between half-max points on each side of the peak
* max: location of y maximum
* min: location of y minimum

In [ ]:
ps = PeakStats('motor', 'noisy_det', calc_derivative_and_stats=True)
RE(
    scan([noisy_det], motor, -1, 4.5, num=30), ps
)

plot_peak_stats(ps)
bec.enable_plots()

## Relative Scan

In [ ]:
help(rel_scan)

In [ ]:
RE(mv(motor, 3))
RE(
    rel_scan([det], motor, -1, 2, num=50)
)

In [ ]:
RE(
    rel_scan([det], motor, -1, 2, num=50)
)

## List Scan

In [ ]:
help(list_scan)

In [ ]:
RE(
    list_scan([det], motor, [-1.4, -1.3, -0.8, -0.7, -0.2, -0.1])
)

## Relative List Scan

In [ ]:
help(rel_list_scan)

In [ ]:
RE(mv(motor, 3))
RE(
    rel_list_scan([det], motor, [-1.4, -1.3, -0.8, -0.7, -0.2, -0.1])
)

## Log Scan

In [ ]:
help(log_scan)

In [ ]:
RE(
    log_scan([det], motor, -1, 1, num=25)
)

### Exercícios

1. Descubra o valor máximo do detector 'det3' e a posição do 'motor3' nesse ponto.
2. Mova o motor 'motor1' e obtenha o valor máximo e mínimo do detector 'det1' utilizando o gráfico PeakStats.
3. Adquira os valores do 'det3' com o motor 'motor3' nos pontos (-8.3, -4.5, -0.45, 0.65, 2.4, 6.2 e 9.8). 
4. Adquira 45 medidas com o detector 'det3' em um intervalo de -1 mm até +1 mm a partir da posição atual do 'motor3'. 

> Dica: Mova o 'motor3' dentro do limite mínimo -10 e limite máximo 10.